# Compare fitted spectra with StarLight

- Author Sylvie Dagoret-Campagne
- Afflilation : IJCLab/IN2P3/CNRS
- Organisation : LSST-DESC
- creation date : 2023-11-27
- 
- conda environmment on my laptop conda_jaxcpu_dsps_py310
- 
- update : 2023-12-10 : applied on run fit of november 27th 2023
- last update : 2024-01-06 : applied on run fit of January 6th 2024
- last update : 2024-01-28 : applied on run fit of January 28th 2024

In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
import re
import pickle 
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import collections
from collections import OrderedDict
import re
import matplotlib.gridspec as gridspec

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  16

## imports

### fitter jaxopt

In [ ]:
from fors2tostellarpopsynthesis.fitters.fitter_jaxopt import (SSP_DATA,mean_spectrum,mean_mags,mean_sfr,ssp_spectrum_fromparam)

In [ ]:
from dsps.cosmology import DEFAULT_COSMOLOGY, age_at_z

In [ ]:
from fors2tostellarpopsynthesis.fitters.fitter_util import plot_params_kde,calc_ratio

### parameters

In [ ]:
from fors2tostellarpopsynthesis.parameters import SSPParametersFit,paramslist_to_dict

### StarLight

In [ ]:
from fors2tostellarpopsynthesis.fors2starlightio import SLDataAcess, flux_norm

In [ ]:
flux_norm

## Configuration

### Constants

In [ ]:
Lyman_lines = [1220., 1030. ,973.,950., 938., 930.]
Balmer_lines = [6562.791,4861.351,4340.4721,4101.740,3970.072,3889.0641,3835.3971]
Paschen_lines = [8750., 12820., 10938.0,10050., 9546.2, 9229.7,9015.3, 8862.89,8750.46,8665.02]
Brackett_lines = [40522.79, 26258.71, 21661.178, 19440., 18179.21]
Pfund_lines = [ 74599.0, 46537.8, 37405.76 , 32969.8, 30400.]
all_Hydrogen_lines = [ Lyman_lines, Balmer_lines, Paschen_lines, Brackett_lines, Pfund_lines]
Color_lines = ["purple", "blue", "green", "red","grey"]
Balmer_thres = 3645.6
Lyman_thres = 911.267
Paschen_thres = 8200.
Brackett_thres = 14580.
Pfund_lines = 22800.
all_Hydrogen_thres = [Lyman_thres , Balmer_thres, Paschen_thres, Brackett_thres, Pfund_lines]

In [ ]:
wl0 = 3645.6

In [ ]:
D4000_red = [4050.,4250] 
D4000_blue = [3750.,3950.]
W_BALMER = [Balmer_thres, Balmer_lines[0]]
W_LYMAN = [Lyman_thres, Lyman_lines[0]]

In [ ]:
def plot_hydrogen_lines(ax):
    nth = len(all_Hydrogen_thres)
    for idx,group_lines in enumerate(all_Hydrogen_lines):
        # select only Lyman and Balmer
        if idx<2:
            color = Color_lines[idx]
            for wl_line in group_lines:
                ax.axvline(wl_line,color=color,lw=0.5)
            if idx< nth:
                ax.axvline(all_Hydrogen_thres[idx],color=color,linestyle=":")
    ax.axvspan(W_LYMAN[0],W_LYMAN[1],facecolor='green', alpha=0.5)
    ax.axvspan(W_BALMER[0],W_BALMER[1],facecolor='yellow', alpha=0.5)

### fitted params

In [ ]:
# current date
#date = "2024-01-06"
date = "2024-01-28"
# previous date
#date = "2023-11-27"
suptitle = f"SED-DSPS-fit ({date})"

In [ ]:
input_path = "fitssp_results"
input_file = f"fitssp_results_{date}.h5"

In [ ]:
#input_path = "fitssp_results"
#input_file = "fitssp_results.h5"

In [ ]:
fullname_input = os.path.join(input_path ,input_file) 

### Functions

In [ ]:
def bluefraction(x,y,wlcut=wl0):

    indexes_blue = np.where(x<wlcut)[0]
    indexes_red =np.where(x>wlcut)[0]

    integ_blue = np.trapz(y[indexes_blue],x[indexes_blue])
    integ_red = np.trapz(y[indexes_red],x[indexes_red])

    fraction_blue = integ_blue /(integ_blue+integ_red)
    fraction_red= integ_red /(integ_blue+integ_red)

    return fraction_blue

## Read StarLight

In [ ]:
sl = SLDataAcess()

In [ ]:
#sl.get_list_subgroup_keys()

In [ ]:
#sl.get_list_of_groupkeys()

In [ ]:
TAG = "SPEC560"
sl.getattribdata_fromgroup(TAG)

## Read Dataframe 

In [ ]:
df = pd.read_hdf(fullname_input)

In [ ]:
df

In [ ]:
N = len(df)

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 4))
df["zobs"].plot(kind='hist', bins=30,range=(0,1.5),facecolor="b",xlabel="z_obs",
                title="redshifts for RXJ 0054.0-2823 : long_gal = 278.66° , lat_gal = -88.60°",ax=ax);


In [ ]:
# create colormap
#cm = plt.cm.bwr(np.linspace(0, 1, N))

## Decode fitted DSPS parameters and plot spectra

In [ ]:
names_fitparams = list(df.columns[8:])

In [ ]:
all_x = []
all_y_nodust = []
all_y_dust = []
the_ymax= 1e-12
all_fractions_blue = []
all_BL = []
all_D4000 = []
all_redshift = []

all_x_sl = []
all_y_sl = []
all_fractions_blue_sl = []
all_BL_sl = []
all_D4000_sl = []
all_redshift_sl = []

# loop on fitted spectra
for idx in range(N):

    # SSP
    row = df.iloc[idx]
    z_obs = row["zobs"]
    all_redshift.append(z_obs)
    
    specname = row["fors2name"]
    list_params = row[names_fitparams].values
    dict_params = paramslist_to_dict(list_params,names_fitparams)
    x,y_nodust,y_dust = ssp_spectrum_fromparam(dict_params,z_obs)

    norm_y_nodust = flux_norm(x,y_nodust,wlcenter=wl0)
    norm_y_dust = flux_norm(x,y_dust,wlcenter=wl0)

    y_nodust /=  norm_y_nodust
    y_dust /=  norm_y_dust
    
    fract_blue = bluefraction(x,y_nodust)
    D4000 = calc_ratio(x,y_nodust)
    DBL = calc_ratio(x,y_nodust,W_LYMAN ,W_BALMER )
    
    
    all_x.append(x)
    all_y_nodust.append(y_nodust) 
    all_y_dust.append(y_dust) 
    all_fractions_blue.append(fract_blue)
    all_BL.append(DBL)
    all_D4000.append(D4000)

    ymax = y_nodust.max()
    the_ymax = max(the_ymax,ymax)
    

    # SL
    
    
    dict_sl = sl.getspectrum_fromgroup(specname)
    x_sl,y_sl = dict_sl["wl"],dict_sl["fnu"]
    attr_dict_sl = sl.getattribdata_fromgroup(specname)
    redshift_sl =  attr_dict_sl['redshift']
    all_redshift_sl.append( redshift_sl)
    

    norm_sl = flux_norm(x_sl,y_sl,wlcenter=wl0)
    y_sl /= norm_sl

    fract_blue_sl = bluefraction(x_sl[:-1],y_sl[:-1])
    D4000_sl = calc_ratio(x_sl[:-1],y_sl[:-1])
    DBL_sl = calc_ratio(x_sl[:-1],y_sl[:-1],W_LYMAN ,W_BALMER )
    

    all_x_sl.append(x_sl)
    all_y_sl.append(y_sl)
    
    all_fractions_blue_sl.append(fract_blue_sl)
    all_BL_sl.append(DBL_sl)
    all_D4000_sl.append(D4000_sl)


ylim_max = the_ymax*2.
ylim_min = ylim_max/1e7

all_fractions_blue = np.array(all_fractions_blue)
all_fractions_blue_sl = np.array(all_fractions_blue_sl)
all_BL = np.array(all_BL)
all_BL_sl = np.array(all_BL_sl)
all_D4000 = np.array(all_D4000)
all_D4000_sl = np.array(all_D4000_sl)

### Check redshift are OK

In [ ]:
# Check redshifts
fig, ax = plt.subplots(1, 1,figsize=(2,2))
ax.scatter(all_redshift_sl,all_redshift)

### Check color index

In [ ]:
bwr_map = plt.get_cmap('bwr')
reversed_map = bwr_map.reversed() 
cNorm = colors.Normalize(0., vmax=1.)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=bwr_map)
all_colors = scalarMap.to_rgba(df["zobs"].values, alpha=1)

fig, ax = plt.subplots(1, 1,figsize=(5,4))
ax.scatter(all_D4000,all_D4000_sl,color=all_colors,alpha=0.5)
ax.set_xlim(1.,2.5)
ax.set_ylim(1.,2.5)
ax.set_xlabel("D4000 in fitted DSPS-Fors2")
ax.set_ylabel("D4000 in fitted SL-Fors2")
cbar=fig.colorbar(scalarMap, ax=ax)
cbar.ax.set_ylabel('redshift')
plt.suptitle(suptitle)

In [ ]:
fig = plt.figure(figsize=(8,6))
axs = fig.subplots(2,2)

for idx, ax in enumerate(np.ravel(axs)):
    if idx==0:
        ax.hist(all_D4000,bins=50,density=True,facecolor="r",label="DSPS")
        ax.hist(all_D4000_sl,bins=50,density=True,facecolor="b",label="SL")
        ax.set_xlabel("D4000")
        ax.legend()
    elif idx==1:
        ax.hist(np.log10(all_BL),bins=50,density=True,facecolor="r")
        ax.hist(np.log10(all_BL_sl),bins=50,density=True,facecolor="b")
        ax.set_xlabel("log10(Balmer/Lyman)")
        ax.set_yscale('log')
    elif idx==2:
        ax.scatter(all_D4000,np.log10(all_BL),marker='o',alpha=0.3,facecolor="r")
        ax.scatter(all_D4000_sl,np.log10(all_BL_sl),marker='o',alpha=0.3,facecolor="b")
        ax.set_xlabel("D4000")
        ax.set_ylabel("log10(Balmer/Lyman)")
    elif idx==3:
        ax.hist(np.log10(all_fractions_blue),density=True,facecolor="r",alpha=0.8)
        ax.hist(np.log10(all_fractions_blue_sl),density=True,facecolor="blue",alpha=0.8)
        ax.set_xlabel("Blue fraction")
#ax.set_title("D4000")
#ax.set_xlabel("D4000")
plt.suptitle(suptitle)
plt.tight_layout()

In [ ]:
D4000MIN = 1.0
D4000MAX = 2.0
BLMIN = 0.8
BLMAX = 4.

In [ ]:
bwr_map = plt.get_cmap('bwr')
reversed_map = bwr_map.reversed() 
#cNorm = colors.Normalize(vmin=np.log10(all_fractions).min(), vmax=np.log10(all_fractions).max())
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=reversed_map)
#all_colors = scalarMap.to_rgba(np.log10(all_fractions), alpha=1)
cNorm = colors.Normalize(vmin=all_D4000.min(), vmax=all_D4000.max())
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=bwr_map)
all_colors = scalarMap.to_rgba(all_D4000, alpha=1)

In [ ]:
#cNorm = colors.Normalize(vmin=np.log10(all_fractions_sl).min(), vmax=np.log10(all_fractions_sl).max())
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=reversed_map)
#all_colors_sl = scalarMap.to_rgba(np.log10(all_fractions_sl), alpha=1)
cNorm = colors.Normalize(vmin=all_D4000_sl.min(), vmax=all_D4000_sl.max())
scalarMap_sl = cmx.ScalarMappable(norm=cNorm, cmap=bwr_map)
all_colors_sl = scalarMap_sl.to_rgba(all_D4000_sl, alpha=1)

In [ ]:
_, (ax1,ax2) = plt.subplots(2, 1,figsize=(10,10))


__= ax1.set_yscale('log') 
__= ax1.set_xscale('log') 

for idx in range(N):
    ax1.plot(all_x[idx],all_y_nodust[idx],'-',color=all_colors[idx])


__= ax1.set_xlim(100.,1e6)
__= ax1.set_ylim(ylim_min ,ylim_max )

ax1.set_xlabel("$\lambda (\\AA)$")
ax1.set_ylabel("$L_\\nu(\lambda)$ relative flux")
ax1.set_title("$f_\\nu(\lambda)$ Fitted spectra")
#ax1.grid()
#ax1.axvline(wl0,color="k")
cbar=fig.colorbar(scalarMap , ax=ax1)
cbar.ax.set_ylabel('D4000')
plot_hydrogen_lines(ax1)



inset_ax = inset_axes(ax1,
                    width="30%", # width = 30% of parent_bbox
                    height="30%", # height : 1 inch
                    loc=4)
inset_ax.hist(all_D4000,bins=20,range=(D4000MIN,D4000MAX),facecolor="g",alpha=0.2)
inset_ax.set_title("D4000",fontsize=12)



__= ax2.set_yscale('log') 
__= ax2.set_xscale('log') 

for idx in range(N):
    ax2.plot(all_x_sl[idx][:-1],all_y_sl[idx][:-1],'-',color=all_colors_sl[idx])
__= ax2.set_xlim(100.,1e6)
__= ax2.set_ylim(ylim_min ,ylim_max )
#ax2.grid()
cbar=fig.colorbar(scalarMap_sl , ax=ax2)
cbar.ax.set_ylabel('D4000')

ax2.set_xlabel("$\lambda (\\AA)$")
ax2.set_ylabel("$L_\\nu(\lambda)$ relative flux")
ax2.set_title("$f_\\nu(\lambda)$ Starlight spectra")
#ax2.axvline(wl0,color="k")
plot_hydrogen_lines(ax2)

inset_ax = inset_axes(ax2,
                    width="30%", # width = 30% of parent_bbox
                    height="30%", # height : 1 inch
                    loc=4)
inset_ax.hist(all_D4000_sl,bins=20,range=(D4000MIN,D4000MAX),facecolor="g",alpha=0.2)
inset_ax.set_title("D4000",fontsize=12)

plt.suptitle(suptitle)
plt.tight_layout()

In [ ]:
_, (ax1,ax2) = plt.subplots(2, 1,figsize=(10,10))
__= ax1.set_yscale('log') 
__= ax1.set_xscale('log') 

for idx in range(N):
    ax1.plot(all_x[idx],all_y_nodust[idx]/(all_x[idx]/wl0)**2,color=all_colors[idx])
__= ax1.set_xlim(100.,1e5)
#__= ax1.set_ylim(1e-12 ,1e-4 )
__= ax1.set_ylim(1e-3 ,1e2 )

#ax1.axvline(wl0,color="k")
ax1.set_xlabel("$\lambda (\\AA)$")
ax1.set_ylabel("$L_\\lambda(\lambda)$ relative flux")
ax1.set_title("$f_\\lambda(\lambda)$ Fitted spectra")
plot_hydrogen_lines(ax1)
#ax1.grid()
cbar=fig.colorbar(scalarMap , ax=ax1)
cbar.ax.set_ylabel('D4000')
plot_hydrogen_lines(ax1)


__= ax2.set_yscale('log') 
__= ax2.set_xscale('log') 

for idx in range(N):
    ax2.plot(all_x_sl[idx][:-1],all_y_sl[idx][:-1]/(all_x_sl[idx][:-1]/wl0)**2,'-',color=all_colors_sl[idx])
__= ax2.set_xlim(100.,1e5)
__= ax2.set_ylim(1e-3 ,1e2 )
ax2.grid()
ax2.set_xlabel("$\lambda (\\AA)$")
ax2.set_ylabel("$L_\\lambda(\lambda)$ relative flux")
ax2.set_title("$f_\\lambda(\lambda)$ Starlight spectra")
#ax2.axvline(wl0,color="k")
cbar=fig.colorbar(scalarMap_sl , ax=ax2)
cbar.ax.set_ylabel('D4000')
plot_hydrogen_lines(ax2)

plt.suptitle(suptitle)
plt.tight_layout()

## Plot Normalised spectra with insets 

## Dependence with redshifts

In [ ]:
bwr_map = plt.get_cmap('bwr')
reversed_map = bwr_map.reversed() 
cNorm = colors.Normalize(0., vmax=1.)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=bwr_map)
all_colors = scalarMap.to_rgba(df["zobs"].values, alpha=1)

In [ ]:
_, (ax1,ax2) = plt.subplots(2, 1,figsize=(10,10))


__= ax1.set_yscale('log') 
__= ax1.set_xscale('log') 

for idx in range(N):
    ax1.plot(all_x[idx],all_y_nodust[idx],'-',color=all_colors[idx])


__= ax1.set_xlim(100.,1e6)
__= ax1.set_ylim(ylim_min ,ylim_max )

ax1.set_xlabel("$\lambda (\\AA)$")
ax1.set_ylabel("$L_\\nu(\lambda)$ relative flux")
ax1.set_title("$f_\\nu(\lambda)$ Fitted spectra")
#ax1.grid()
#ax1.axvline(wl0,color="k")
cbar=fig.colorbar(scalarMap , ax=ax1)
cbar.ax.set_ylabel('redshift')
plot_hydrogen_lines(ax1)

#inset D4000
#left, bottom, width, height = [0.16, 0.66, 0.15, 0.2]
#ax3 = fig.add_axes([left, bottom, width, height])
#ax3.hist(all_D4000,bins=20,range=(D4000MIN,D4000MAX),facecolor="g",alpha=0.2)
#ax3.set_xlabel("D4000",fontsize=8)


__= ax2.set_yscale('log') 
__= ax2.set_xscale('log') 

for idx in range(N):
    ax2.plot(all_x_sl[idx][:-1],all_y_sl[idx][:-1],'-',color=all_colors[idx])
__= ax2.set_xlim(100.,1e6)
__= ax2.set_ylim(ylim_min ,ylim_max )
#ax2.grid()
cbar=fig.colorbar(scalarMap, ax=ax2)
cbar.ax.set_ylabel('redshift')

ax2.set_xlabel("$\lambda (\\AA)$")
ax2.set_ylabel("$L_\\nu(\lambda)$ relative flux")
ax2.set_title("$f_\\nu(\lambda)$ Starlight spectra")
#ax2.axvline(wl0,color="k")
plot_hydrogen_lines(ax2)

plt.suptitle(suptitle)
plt.tight_layout()